## The Data Preparation Part

Data Sources:  
1) Kaggle Competition [Predict Future Sales](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview), kindly provided by 1C Company, one of the largest Russian software firms  
2) Moscow Airport METAR (Meteorological Aerodrome Report) data, provided by [rp5.ru](https://rp5.ru/Weather_in_Moscow,_Russia)

First import any lib, we will install Pandasql via PyPl on Databricks

Python Lib Ref:  
Pandasql: https://pypi.org/project/pandasql/

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#pip install -U pandasql
from pandasql import sqldf



Now load the sales data provided by 1C Company

In [6]:
df_1CsalesT = pd.read_csv("/dbfs/FileStore/tables/sales_train_v2.csv")

df_1C_items = pd.read_csv("/dbfs/FileStore/tables/items.csv")

df_1C_itemCat = pd.read_csv("/dbfs/FileStore/tables/item_categories.csv")

df_1C_shop = pd.read_csv("/dbfs/FileStore/tables/shops.csv")

We will preview the data for sales and shops

In [8]:
print(df_1CsalesT.shape)

print(df_1CsalesT.head(10))

print(df_1C_shop.shape)

print(df_1C_shop.head(30))


(2935849, 6)
 date date_block_num shop_id item_id item_price item_cnt_day
0 02.01.2013 0 59 22154 999.00 1.0
1 03.01.2013 0 25 2552 899.00 1.0
2 05.01.2013 0 25 2552 899.00 -1.0
3 06.01.2013 0 25 2554 1709.05 1.0
4 15.01.2013 0 25 2555 1099.00 1.0
5 10.01.2013 0 25 2564 349.00 1.0
6 02.01.2013 0 25 2565 549.00 1.0
7 04.01.2013 0 25 2572 239.00 1.0
8 11.01.2013 0 25 2572 299.00 1.0
9 03.01.2013 0 25 2573 299.00 3.0
(60, 2)
 shop_name shop_id
0 !Якутск Орджоникидзе, 56 фран 0
1 !Якутск ТЦ "Центральный" фран 1
2 Адыгея ТЦ "Мега" 2
3 Балашиха ТРК "Октябрь-Киномир" 3
4 Волжский ТЦ "Волга Молл" 4
5 Вологда ТРЦ "Мармелад" 5
6 Воронеж (Плехановская, 13) 6
7 Воронеж ТРЦ "Максимир" 7
8 Воронеж ТРЦ Сити-Парк "Град" 8
9 Выездная Торговля 9
10 Жуковский ул. Чкалова 39м? 10
11 Жуковский ул. Чкалова 39м² 11
12 Интернет-магазин ЧС 12
13 Казань ТЦ "Бехетле" 13
14 Казань ТЦ "ПаркХаус" II 14
15 Калуга ТРЦ "XXI век" 15
16 Коломна ТЦ "Рио" 16
17 Красноярск ТЦ "Взлетка Плаза" 17
18 Красноярск ТЦ "Июнь" 18
19 Курск ТЦ "Пушкинский" 19
20 Москва "Распродажа" 20
21 Москва МТРЦ "Афи Молл" 21
22 Москва Магазин С21 22
23 Москва ТК "Буденовский" (пав.А2) 23
24 Москва ТК "Буденовский" (пав.К7) 24
25 Москва ТРК "Атриум" 25
26 Москва ТЦ "Ареал" (Беляево) 26
27 Москва ТЦ "МЕГА Белая Дача II" 27
28 Москва ТЦ "МЕГА Теплый Стан" II 28
29 Москва ТЦ "Новый век" (Новокосино) 29

We should find the revenue for each sales record by multiplying item_price and item_cnt_day, for df_1CsalesT, create a new column to store it, and it will be used latter. This step will be long due to the large set of data.

In [10]:
#df_1CsalesT['Revenue'] = df_1CsalesT.apply(lambda row: row.item_price * row.item_cnt_day, axis=1)
df_1CsalesT['Revenue'] = df_1CsalesT ['item_price'] * df_1CsalesT ['item_cnt_day']
df_1CsalesT.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Revenue
0,02.01.2013,0,59,22154,999.00,1.0,999.00
1,03.01.2013,0,25,2552,899.00,1.0,899.00
2,05.01.2013,0,25,2552,899.00,-1.0,-899.00
3,06.01.2013,0,25,2554,1709.05,1.0,1709.05
4,15.01.2013,0,25,2555,1099.00,1.0,1099.00
5,10.01.2013,0,25,2564,349.00,1.0,349.00
6,02.01.2013,0,25,2565,549.00,1.0,549.00
7,04.01.2013,0,25,2572,239.00,1.0,239.00
8,11.01.2013,0,25,2572,299.00,1.0,299.00
9,03.01.2013,0,25,2573,299.00,3.0,897.00


In [11]:
#Let's check again on the size, see how it looks comparing to original
df_1CsalesT.shape

Out[5]: (2935849, 7)

I have tried to do translation wth Google Translate: https://pypi.org/project/googletrans/, but it sometimes does not work. Anyway, I found "Москва" is "Moscow", and since we only care about the 1C shops in Moscow, we will filter them out. In addition, I found "Цифровой склад 1С-Онлайн,55" is "Digital warehouse 1C-Online", with shop_id of ''55. We will look at it as well, because colder weather might boost online sales

In [13]:

df_Moscow_shop = df_1C_shop[df_1C_shop['shop_name'].str.contains("Москва")]

print(df_Moscow_shop)

type(df_Moscow_shop )

shop_name shop_id
20 Москва "Распродажа" 20
21 Москва МТРЦ "Афи Молл" 21
22 Москва Магазин С21 22
23 Москва ТК "Буденовский" (пав.А2) 23
24 Москва ТК "Буденовский" (пав.К7) 24
25 Москва ТРК "Атриум" 25
26 Москва ТЦ "Ареал" (Беляево) 26
27 Москва ТЦ "МЕГА Белая Дача II" 27
28 Москва ТЦ "МЕГА Теплый Стан" II 28
29 Москва ТЦ "Новый век" (Новокосино) 29
30 Москва ТЦ "Перловский" 30
31 Москва ТЦ "Семеновский" 31
32 Москва ТЦ "Серебряный Дом" 32
Out[6]: pandas.core.frame.DataFrame

In [14]:
# Get the shop IDs into a list

ls_shopID = df_Moscow_shop['shop_id'].tolist()

print(ls_shopID)

[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]

Then we can select sales data base on that list of shop IDs, this will reduce the total amount of records being analyzed as well.

In [16]:
#Then we can select sales data base on that list of shop IDs.
df_Moscow_sales = df_1CsalesT[df_1CsalesT['shop_id'].isin(ls_shopID)]

print(df_Moscow_sales.shape)

print(df_Moscow_sales)

(996636, 7)
 date date_block_num shop_id ... item_price item_cnt_day Revenue
1 03.01.2013 0 25 ... 899.00 1.0 899.00
2 05.01.2013 0 25 ... 899.00 -1.0 -899.00
3 06.01.2013 0 25 ... 1709.05 1.0 1709.05
4 15.01.2013 0 25 ... 1099.00 1.0 1099.00
5 10.01.2013 0 25 ... 349.00 1.0 349.00
6 02.01.2013 0 25 ... 549.00 1.0 549.00
7 04.01.2013 0 25 ... 239.00 1.0 239.00
8 11.01.2013 0 25 ... 299.00 1.0 299.00
9 03.01.2013 0 25 ... 299.00 3.0 897.00
10 03.01.2013 0 25 ... 399.00 2.0 798.00
11 05.01.2013 0 25 ... 399.00 1.0 399.00
12 07.01.2013 0 25 ... 399.00 1.0 399.00
13 08.01.2013 0 25 ... 399.00 2.0 798.00
14 10.01.2013 0 25 ... 399.00 1.0 399.00
15 11.01.2013 0 25 ... 399.00 2.0 798.00
16 13.01.2013 0 25 ... 399.00 1.0 399.00
17 16.01.2013 0 25 ... 399.00 1.0 399.00
18 26.01.2013 0 25 ... 399.00 1.0 399.00
19 27.01.2013 0 25 ... 399.00 1.0 399.00
20 09.01.2013 0 25 ... 279.00 1.0 279.00
21 16.01.2013 0 25 ... 299.00 1.0 299.00
22 27.01.2013 0 25 ... 299.00 1.0 299.00
23 27.01.2013 0 25 ... 279.00 1.0 279.00
24 29.01.2013 0 25 ... 279.00 1.0 279.00
25 27.01.2013 0 25 ... 1699.00 1.0 1699.00
26 06.01.2013 0 25 ... 1708.95 1.0 1708.95
27 26.01.2013 0 25 ... 299.00 1.0 299.00
28 02.01.2013 0 25 ... 299.00 1.0 299.00
29 06.01.2013 0 25 ... 1649.00 1.0 1649.00
30 07.01.2013 0 25 ... 1799.00 1.0 1799.00
... ... ... ... ... ... ... ...
2935819 16.10.2015 33 25 ... 1799.00 1.0 1799.00
2935820 22.10.2015 33 25 ... 799.00 1.0 799.00
2935821 22.10.2015 33 25 ... 899.50 1.0 899.50
2935822 17.10.2015 33 25 ... 2310.00 1.0 2310.00
2935823 26.10.2015 33 25 ... 299.00 1.0 299.00
2935824 14.10.2015 33 25 ... 999.00 1.0 999.00
2935825 23.10.2015 33 25 ... 299.00 1.0 299.00
2935826 04.10.2015 33 25 ... 599.00 1.0 599.00
2935827 18.10.2015 33 25 ... 349.00 1.0 349.00
2935828 21.10.2015 33 25 ... 479.00 1.0 479.00
2935829 03.10.2015 33 25 ... 599.00 1.0 599.00
2935830 11.10.2015 33 25 ... 299.00 1.0 299.00
2935831 22.10.2015 33 25 ... 299.00 1.0 299.00
2935832 26.10.2015 33 25 ... 299.00 1.0 299.00
2935833 22.10.2015 33 25 ... 349.00 1.0 349.00
2935834 29.10.2015 33 25 ... 298.00 1.0 298.00
2935835 18.10.2015 33 25 ... 349.00 1.0 349.00
2935836 22.10.2015 33 25 ... 349.00 1.0 349.00
2935837 24.10.2015 33 25 ... 249.00 1.0 249.00
2935838 17.10.2015 33 25 ... 349.00 1.0 349.00
2935839 24.10.2015 33 25 ... 399.00 1.0 399.00
2935840 31.10.2015 33 25 ... 299.00 1.0 299.00
2935841 11.10.2015 33 25 ... 349.00 1.0 349.00
2935842 10.10.2015 33 25 ... 749.00 1.0 749.00
2935843 09.10.2015 33 25 ... 299.00 1.0 299.00
2935844 10.10.2015 33 25 ... 299.00 1.0 299.00
2935845 09.10.2015 33 25 ... 299.00 1.0 299.00
2935846 14.10.2015 33 25 ... 349.00 1.0 349.00
2935847 22.10.2015 33 25 ... 299.00 1.0 299.00
2935848 03.10.2015 33 25 ... 299.00 1.0 299.00

[996636 rows x 7 columns]

We also get the data set for online sales

In [18]:
df_OL_sales = df_1CsalesT[df_1CsalesT['shop_id'].isin(['55'])]

print(df_OL_sales.shape)

print(df_OL_sales)

(34769, 7)
 date date_block_num shop_id ... item_price item_cnt_day Revenue
503292 06.05.2013 4 55 ... 119.0 1.0 119.0
503293 10.05.2013 4 55 ... 119.0 3.0 357.0
503294 12.05.2013 4 55 ... 119.0 3.0 357.0
503295 14.05.2013 4 55 ... 119.0 1.0 119.0
503296 16.05.2013 4 55 ... 119.0 2.0 238.0
503297 18.05.2013 4 55 ... 119.0 1.0 119.0
503298 21.05.2013 4 55 ... 119.0 1.0 119.0
503299 22.05.2013 4 55 ... 119.0 1.0 119.0
503300 05.05.2013 4 55 ... 119.0 3.0 357.0
503301 24.05.2013 4 55 ... 119.0 1.0 119.0
503302 27.05.2013 4 55 ... 119.0 1.0 119.0
503303 08.05.2013 4 55 ... 399.0 1.0 399.0
503304 19.05.2013 4 55 ... 899.0 1.0 899.0
503305 20.05.2013 4 55 ... 899.0 2.0 1798.0
503306 23.05.2013 4 55 ... 899.0 1.0 899.0
503307 24.05.2013 4 55 ... 899.0 1.0 899.0
503308 26.05.2013 4 55 ... 899.0 1.0 899.0
503309 27.05.2013 4 55 ... 899.0 1.0 899.0
503310 25.05.2013 4 55 ... 119.0 1.0 119.0
503311 04.05.2013 4 55 ... 119.0 2.0 238.0
503312 29.05.2013 4 55 ... 399.0 2.0 798.0
503313 24.05.2013 4 55 ... 399.0 1.0 399.0
503314 22.05.2013 4 55 ... 59.0 1.0 59.0
503315 05.05.2013 4 55 ... 199.0 1.0 199.0
503316 19.05.2013 4 55 ... 199.0 1.0 199.0
503317 21.05.2013 4 55 ... 199.0 1.0 199.0
503318 18.05.2013 4 55 ... 149.0 1.0 149.0
503319 04.05.2013 4 55 ... 599.0 1.0 599.0
503320 06.05.2013 4 55 ... 599.0 1.0 599.0
503321 10.05.2013 4 55 ... 599.0 1.0 599.0
... ... ... ... ... ... ... ...
2905537 19.10.2015 33 55 ... 6601.0 1.0 6601.0
2905538 27.10.2015 33 55 ... 6724.0 1.0 6724.0
2905539 22.10.2015 33 55 ... 25.0 1.0 25.0
2905540 23.10.2015 33 55 ... 25.0 1.0 25.0
2905541 24.10.2015 33 55 ... 249.0 1.0 249.0
2905542 17.10.2015 33 55 ... 599.0 7.0 4193.0
2905543 25.10.2015 33 55 ... 249.0 1.0 249.0
2905544 25.10.2015 33 55 ... 249.0 1.0 249.0
2905545 03.10.2015 33 55 ... 419.0 1.0 419.0
2905546 01.10.2015 33 55 ... 249.0 1.0 249.0
2905547 01.10.2015 33 55 ... 599.0 1.0 599.0
2905548 02.10.2015 33 55 ... 599.0 1.0 599.0
2905549 03.10.2015 33 55 ... 599.0 1.0 599.0
2905550 24.10.2015 33 55 ... 375.0 2.0 750.0
2905551 26.10.2015 33 55 ... 375.0 3.0 1125.0
2905552 02.10.2015 33 55 ... 49.0 1.0 49.0
2905553 14.10.2015 33 55 ... 49.0 1.0 49.0
2905554 24.10.2015 33 55 ... 187.0 1.0 187.0
2905555 25.10.2015 33 55 ... 187.0 1.0 187.0
2905556 25.10.2015 33 55 ... 330.0 1.0 330.0
2905557 26.10.2015 33 55 ... 330.0 1.0 330.0
2905558 17.10.2015 33 55 ... 199.0 1.0 199.0
2905559 11.10.2015 33 55 ... 999.0 1.0 999.0
2905560 24.10.2015 33 55 ... 999.0 1.0 999.0
2905561 12.10.2015 33 55 ... 499.0 1.0 499.0
2905562 19.10.2015 33 55 ... 499.0 1.0 499.0
2905563 30.10.2015 33 55 ... 499.0 1.0 499.0
2905564 14.10.2015 33 55 ... 499.0 1.0 499.0
2905565 24.10.2015 33 55 ... 249.0 1.0 249.0
2905566 24.10.2015 33 55 ... 249.0 1.0 249.0

[34769 rows x 7 columns]

We can find the sum of revenues for total sales, sales in Moscow, and online sales, see how much they are

In [20]:
total_sales = df_1CsalesT['Revenue'].sum() 
msc_sales = df_Moscow_sales['Revenue'].sum() 
ol_sales = df_OL_sales['Revenue'].sum() 

print('Total 1C Sales of all (in MRub): ', total_sales/1000000)
print('Moscow Sales (in MRub): ', msc_sales/1000000)
print('Online Sales (in MRub):', ol_sales/1000000)

Total 1C Sales of all (in MRub): 3398.9263445699917
Moscow Sales (in MRub): 1132.706329150001
Online Sales (in MRub): 49.792061499998646

From above, total sales from 2013-01 to 2015-10 is about 3398.9 MRub, and revenue from shops in Moscow is around 1/3 of that, at 1132.7 MRub. Interestingly, the sales from Online is only about 49.8 MRub, not very significant contribution. In this case, we will just focus on the sales in Moscow.

Now we want to check the month-year coverage of given 1C sales data for Moscow. We can observe that month-year were from 01.2013 to 10.2015.  
We will be using Pandas SQL: https://pypi.org/project/pandasql/

In [23]:
pysqldf = lambda q: sqldf(q, globals())

df_dates = pysqldf('''SELECT distinct substr(date, -7) FROM df_Moscow_sales ''')

print(df_dates)


substr(date, -7)
0 01.2013
1 02.2013
2 03.2013
3 04.2013
4 05.2013
5 06.2013
6 07.2013
7 08.2013
8 09.2013
9 10.2013
10 11.2013
11 12.2013
12 01.2014
13 02.2014
14 03.2014
15 04.2014
16 05.2014
17 06.2014
18 07.2014
19 08.2014
20 09.2014
21 10.2014
22 11.2014
23 12.2014
24 01.2015
25 02.2015
26 03.2015
27 04.2015
28 05.2015
29 06.2015
30 07.2015
31 08.2015
32 09.2015
33 10.2015

For each date, we want to get the daily item count and daily sales revenue, fot the 1C shops in Moscow

In [25]:
df_MSC_sales = pysqldf('''SELECT date,
                                 sum(Revenue) AS DailyRev,
                                 sum(item_cnt_day) AS ItemsSold
                          FROM df_Moscow_sales
                          GROUP BY date
                          ORDER BY date''')

df_MSC_sales['date'] =  pd.to_datetime(df_MSC_sales['date'], format='%d.%m.%Y')

df_MSC_sales = df_MSC_sales.sort_values(by='date')

print(df_MSC_sales.dtypes)
print(df_MSC_sales.shape)
print(df_MSC_sales)


date datetime64[ns]
DailyRev float64
ItemsSold float64
dtype: object
(1034, 3)
 date DailyRev ItemsSold
0 2013-01-01 296395.00 464.0
34 2013-01-02 2276285.23 2999.0
68 2013-01-03 1798752.11 2433.0
102 2013-01-04 1550643.00 2234.0
136 2013-01-05 1537602.00 2162.0
170 2013-01-06 1589978.00 2129.0
204 2013-01-07 1219763.23 1811.0
238 2013-01-08 1196487.00 1709.0
272 2013-01-09 782693.00 1114.0
306 2013-01-10 822479.50 1229.0
340 2013-01-11 1123552.00 1497.0
374 2013-01-12 1339859.00 1975.0
408 2013-01-13 1045654.00 1651.0
442 2013-01-14 679710.00 914.0
476 2013-01-15 957684.50 1249.0
510 2013-01-16 804121.00 1032.0
544 2013-01-17 769743.00 1135.0
578 2013-01-18 932859.60 1370.0
612 2013-01-19 1370661.00 2123.0
646 2013-01-20 1263469.00 1786.0
680 2013-01-21 661074.00 934.0
714 2013-01-22 720297.00 1029.0
748 2013-01-23 718970.00 1092.0
782 2013-01-24 746888.00 1227.0
816 2013-01-25 1146252.87 1727.0
850 2013-01-26 1473781.00 2356.0
884 2013-01-27 1200931.00 1865.0
918 2013-01-28 646742.00 971.0
952 2013-01-29 626688.00 998.0
983 2013-01-30 752470.00 1160.0
... ... ... ...
63 2015-10-02 1407607.00 1282.0
97 2015-10-03 2118840.00 1824.0
131 2015-10-04 1737621.00 1580.0
165 2015-10-05 497963.00 433.0
199 2015-10-06 397528.00 466.0
233 2015-10-07 999658.00 717.0
267 2015-10-08 702621.00 586.0
301 2015-10-09 834100.00 773.0
335 2015-10-10 937323.00 883.0
369 2015-10-11 710319.00 791.0
403 2015-10-12 488217.00 469.0
437 2015-10-13 700467.00 836.0
471 2015-10-14 588063.00 641.0
505 2015-10-15 476048.00 577.0
539 2015-10-16 707385.00 727.0
573 2015-10-17 791059.00 821.0
607 2015-10-18 575139.00 685.0
641 2015-10-19 461708.00 455.0
675 2015-10-20 1068523.00 603.0
709 2015-10-21 1071100.00 642.0
743 2015-10-22 786345.00 569.0
777 2015-10-23 2285143.00 1106.0
811 2015-10-24 1410905.00 1035.0
845 2015-10-25 1030561.00 812.0
879 2015-10-26 645174.00 419.0
913 2015-10-27 606041.00 471.0
947 2015-10-28 525524.00 412.0
978 2015-10-29 590557.00 598.0
1009 2015-10-30 932222.00 911.0
1031 2015-10-31 919933.00 965.0

[1034 rows x 3 columns]

Next, we load the weather data of Moscow, from here https://rp5.ru/Weather_archive_in_Sheremetyevo_(airport),_METAR

In [27]:
#Load the Moscow Temp data, note some other columns have been removed, with only DateTime and AirTemp left
df_MoscowTemp = pd.read_csv("/dbfs/FileStore/tables/Moscow_SVO_Temp1315.csv")

df_MoscowTemp.shape

Out[13]: (38379, 2)

In [28]:
df_MoscowTemp.head()




,Local_DateTime,AirTemp
0,30.12.2015 23:30,-8.0
1,30.12.2015 23:00,-8.0
2,30.12.2015 22:30,-8.0
3,30.12.2015 22:00,-7.0
4,30.12.2015 21:00,-7.0


We add a column that is date only, because we will aggreate base on each date. We used 'date' so it will be the same column name as in sales data.

In [30]:
df_MoscowDateTemp = df_MoscowTemp.copy()

df_MoscowDateTemp["date"] = df_MoscowDateTemp.Local_DateTime.str[:10]                                 

df_MoscowDateTemp['date'] =  pd.to_datetime(df_MoscowDateTemp['date'], format='%d.%m.%Y')


df_MoscowDateTemp.head()

,Local_DateTime,AirTemp,date
0,30.12.2015 23:30,-8.0,2015-12-30
1,30.12.2015 23:00,-8.0,2015-12-30
2,30.12.2015 22:30,-8.0,2015-12-30
3,30.12.2015 22:00,-7.0,2015-12-30
4,30.12.2015 21:00,-7.0,2015-12-30


We should check that the year-month covers what we are looking for

In [32]:
df_wea_dates = pysqldf('''SELECT distinct substr(date, 0,8) FROM df_MoscowDateTemp ''')

print(df_wea_dates)



substr(date, 0,8)
0 2015-12
1 2015-11
2 2015-10
3 2015-09
4 2015-08
5 2015-07
6 2015-06
7 2015-05
8 2015-04
9 2015-03
10 2015-02
11 2015-01
12 2014-12
13 2014-11
14 2014-10
15 2014-09
16 2014-08
17 2014-07
18 2014-06
19 2014-05
20 2014-04
21 2014-03
22 2014-02
23 2014-01
24 2013-12
25 2013-11
26 2013-10
27 2013-09
28 2013-08
29 2013-07
30 2013-06
31 2013-05
32 2013-04
33 2013-03
34 2013-02
35 2013-01
36 2012-12

Find average temp for each date

In [34]:
df_DateTemp = pysqldf('''SELECT date, AVG([AirTemp]) AS AvgTemp
                         FROM df_MoscowDateTemp
                         GROUP BY Date
                         ORDER BY Date''')

df_DateTemp['date'] =  pd.to_datetime(df_DateTemp['date'])

print(df_DateTemp.shape)
print (df_DateTemp)

(1095, 2)
 date AvgTemp
0 2012-12-31 -6.750000
1 2013-01-01 -1.833333
2 2013-01-02 -0.250000
3 2013-01-03 -1.958333
4 2013-01-04 -0.083333
5 2013-01-05 -1.833333
6 2013-01-06 -5.083333
7 2013-01-07 -5.500000
8 2013-01-08 -8.291667
9 2013-01-09 -10.916667
10 2013-01-10 -11.666667
11 2013-01-11 -13.000000
12 2013-01-12 -13.000000
13 2013-01-13 -13.440000
14 2013-01-14 -8.625000
15 2013-01-15 -7.500000
16 2013-01-16 -8.291667
17 2013-01-17 -9.208333
18 2013-01-18 -12.750000
19 2013-01-19 -13.875000
20 2013-01-20 -15.500000
21 2013-01-21 -12.791667
22 2013-01-22 -14.958333
23 2013-01-23 -13.291667
24 2013-01-24 -11.083333
25 2013-01-25 -12.583333
26 2013-01-26 -14.208333
27 2013-01-27 -14.500000
28 2013-01-28 -12.875000
29 2013-01-29 -7.250000
... ... ...
1065 2015-12-01 0.371429
1066 2015-12-02 0.694444
1067 2015-12-03 -1.473684
1068 2015-12-04 0.195122
1069 2015-12-05 2.439024
1070 2015-12-06 4.475000
1071 2015-12-07 5.179487
1072 2015-12-08 2.170732
1073 2015-12-09 -0.205128
1074 2015-12-10 0.000000
1075 2015-12-11 0.925926
1076 2015-12-12 -1.108108
1077 2015-12-13 0.000000
1078 2015-12-14 -1.432432
1079 2015-12-15 -2.000000
1080 2015-12-16 -2.564103
1081 2015-12-17 -5.631579
1082 2015-12-18 -4.292683
1083 2015-12-19 1.410256
1084 2015-12-20 3.487179
1085 2015-12-21 7.973684
1086 2015-12-22 5.416667
1087 2015-12-23 4.324324
1088 2015-12-24 4.634146
1089 2015-12-25 1.390244
1090 2015-12-26 1.813953
1091 2015-12-27 -3.023256
1092 2015-12-28 -6.789474
1093 2015-12-29 -9.540541
1094 2015-12-30 -7.289474

[1095 rows x 2 columns]

Now we can combine the two data sets, [df_MSC_sales] and [df_DateTemp], w.r.t. their ['date']

In [36]:
df_SalesTemp = pysqldf('''SELECT A.date,
                                 A.DailyRev,
                                 A.ItemsSold,
                                 B.AvgTemp
                         FROM df_MSC_sales AS A
                         INNER JOIN df_DateTemp AS B
                         ON A.date = B.date
                         ORDER BY A.date''')

df_SalesTemp['date'] =  pd.to_datetime(df_SalesTemp['date'])

print(list(df_SalesTemp.columns)) 
print(df_SalesTemp.shape)
print(df_SalesTemp)

print(df_SalesTemp.dtypes)
                           


['date', 'DailyRev', 'ItemsSold', 'AvgTemp']
(1034, 4)
 date DailyRev ItemsSold AvgTemp
0 2013-01-01 296395.00 464.0 -1.833333
1 2013-01-02 2276285.23 2999.0 -0.250000
2 2013-01-03 1798752.11 2433.0 -1.958333
3 2013-01-04 1550643.00 2234.0 -0.083333
4 2013-01-05 1537602.00 2162.0 -1.833333
5 2013-01-06 1589978.00 2129.0 -5.083333
6 2013-01-07 1219763.23 1811.0 -5.500000
7 2013-01-08 1196487.00 1709.0 -8.291667
8 2013-01-09 782693.00 1114.0 -10.916667
9 2013-01-10 822479.50 1229.0 -11.666667
10 2013-01-11 1123552.00 1497.0 -13.000000
11 2013-01-12 1339859.00 1975.0 -13.000000
12 2013-01-13 1045654.00 1651.0 -13.440000
13 2013-01-14 679710.00 914.0 -8.625000
14 2013-01-15 957684.50 1249.0 -7.500000
15 2013-01-16 804121.00 1032.0 -8.291667
16 2013-01-17 769743.00 1135.0 -9.208333
17 2013-01-18 932859.60 1370.0 -12.750000
18 2013-01-19 1370661.00 2123.0 -13.875000
19 2013-01-20 1263469.00 1786.0 -15.500000
20 2013-01-21 661074.00 934.0 -12.791667
21 2013-01-22 720297.00 1029.0 -14.958333
22 2013-01-23 718970.00 1092.0 -13.291667
23 2013-01-24 746888.00 1227.0 -11.083333
24 2013-01-25 1146252.87 1727.0 -12.583333
25 2013-01-26 1473781.00 2356.0 -14.208333
26 2013-01-27 1200931.00 1865.0 -14.500000
27 2013-01-28 646742.00 971.0 -12.875000
28 2013-01-29 626688.00 998.0 -7.250000
29 2013-01-30 752470.00 1160.0 -4.791667
... ... ... ... ...
1004 2015-10-02 1407607.00 1282.0 11.307692
1005 2015-10-03 2118840.00 1824.0 12.052632
1006 2015-10-04 1737621.00 1580.0 8.390244
1007 2015-10-05 497963.00 433.0 11.170732
1008 2015-10-06 397528.00 466.0 4.878049
1009 2015-10-07 999658.00 717.0 0.142857
1010 2015-10-08 702621.00 586.0 0.970588
1011 2015-10-09 834100.00 773.0 -0.764706
1012 2015-10-10 937323.00 883.0 1.000000
1013 2015-10-11 710319.00 791.0 1.926829
1014 2015-10-12 488217.00 469.0 3.756098
1015 2015-10-13 700467.00 836.0 3.781250
1016 2015-10-14 588063.00 641.0 2.454545
1017 2015-10-15 476048.00 577.0 5.464286
1018 2015-10-16 707385.00 727.0 3.800000
1019 2015-10-17 791059.00 821.0 4.365854
1020 2015-10-18 575139.00 685.0 6.571429
1021 2015-10-19 461708.00 455.0 5.071429
1022 2015-10-20 1068523.00 603.0 0.411765
1023 2015-10-21 1071100.00 642.0 -1.058824
1024 2015-10-22 786345.00 569.0 0.966667
1025 2015-10-23 2285143.00 1106.0 2.088235
1026 2015-10-24 1410905.00 1035.0 4.048780
1027 2015-10-25 1030561.00 812.0 5.285714
1028 2015-10-26 645174.00 419.0 5.428571
1029 2015-10-27 606041.00 471.0 4.540541
1030 2015-10-28 525524.00 412.0 1.128205
1031 2015-10-29 590557.00 598.0 0.151515
1032 2015-10-30 932222.00 911.0 0.529412
1033 2015-10-31 919933.00 965.0 -1.375000

[1034 rows x 4 columns]
date datetime64[ns]
DailyRev float64
ItemsSold float64
AvgTemp float64
dtype: object

We can find the daily averages and standard deviations for all 3 series

In [38]:
print('The daily averages are:', df_SalesTemp.mean())

print('The daily standard deviations are:', df_SalesTemp.std())

The daily averages are: DailyRev 1.095461e+06
ItemsSold 1.234406e+03
AvgTemp 6.950757e+00
dtype: float64
The daily standard deviations are: DailyRev 747742.464884
ItemsSold 570.724269
AvgTemp 10.367651
dtype: float64

We can save this combined data set to a CSV and export for latter use.

In [40]:
df_SalesTemp.to_csv (r'/dbfs/FileStore/tables/MSC_SalesTemp.csv', index = None, header=True)



Check if above works

In [42]:
df_MSC_SalesTemp = pd.read_csv("/dbfs/FileStore/tables/MSC_SalesTemp.csv")

print(df_MSC_SalesTemp.shape)
print(df_MSC_SalesTemp.head())

(1034, 4)
 date DailyRev ItemsSold AvgTemp
0 2013-01-01 296395.00 464.0 -1.833333
1 2013-01-02 2276285.23 2999.0 -0.250000
2 2013-01-03 1798752.11 2433.0 -1.958333
3 2013-01-04 1550643.00 2234.0 -0.083333
4 2013-01-05 1537602.00 2162.0 -1.833333

We also generate a monthly sales temp data set for latter use

In [44]:
df_MonthSalesTemp = pysqldf('''SELECT  distinct 
                                 substr(date, 0,8) AS Month,
                                 sum(DailyRev)/1000 AS MonthKRub,
                                 sum(ItemsSold) AS MonItemSold,
                                 avg(AvgTemp) AS MonAvgTemp
                         FROM df_SalesTemp 
                         GROUP BY Month
                         ORDER BY Month''')


print(list(df_MonthSalesTemp.columns)) 
print(df_MonthSalesTemp.shape)
print(df_SalesTemp.dtypes)

print(df_MonthSalesTemp)



['Month', 'MonthKRub', 'MonItemSold', 'MonAvgTemp']
(34, 4)
date datetime64[ns]
DailyRev float64
ItemsSold float64
AvgTemp float64
dtype: object
 Month MonthKRub MonItemSold MonAvgTemp
0 2013-01 32739.59204 47495.0 -9.156667
1 2013-02 33098.88780 46744.0 -4.084735
2 2013-03 38682.50245 54830.0 -7.410921
3 2013-04 26048.29295 41348.0 5.498007
4 2013-05 23351.54330 38878.0 16.610582
5 2013-06 25718.10220 42386.0 19.743669
6 2013-07 23483.18459 40787.0 18.802595
7 2013-08 25248.43192 43818.0 18.296476
8 2013-09 42016.35750 49326.0 10.109722
9 2013-10 34760.34540 43529.0 6.241938
10 2013-11 45329.48700 46071.0 3.665278
11 2013-12 68725.43000 65128.0 -2.237903
12 2014-01 31468.82490 41374.0 -9.503740
13 2014-02 33846.50644 38571.0 -2.290423
14 2014-03 34848.51350 39825.0 2.116945
15 2014-04 27680.04450 34753.0 6.219457
16 2014-05 29891.69530 33574.0 15.543074
17 2014-06 27191.56600 33695.0 15.622930
18 2014-07 23648.17790 29879.0 20.424123
19 2014-08 28552.45567 34517.0 18.771981
20 2014-09 37286.04490 33918.0 11.954893
21 2014-10 37794.24270 37181.0 3.054868
22 2014-11 49006.77465 39963.0 -1.893893
23 2014-12 78800.08400 59487.0 -4.485144
24 2015-01 40756.06222 38014.0 -4.964412
25 2015-02 28896.61545 28849.0 -2.678947
26 2015-03 29484.52500 28317.0 1.408871
27 2015-04 30803.99100 28437.0 5.444805
28 2015-05 29865.00400 26058.0 13.821401
29 2015-06 22508.72150 23312.0 17.391813
30 2015-07 19454.90129 21508.0 17.827327
31 2015-08 17101.32532 19179.0 17.195104
32 2015-09 26598.86476 21753.0 13.619902
33 2015-10 28019.23100 23872.0 3.749953

We can find the monthly averages with standard deviations for all 3 series

In [46]:

print('The monthly averages are:', df_MonthSalesTemp.mean())

print('The monthly standard deviations are:', df_MonthSalesTemp.std())


The monthly averages are: MonthKRub 33314.892034
MonItemSold 37540.470588
MonAvgTemp 6.894968
dtype: float64
The monthly standard deviations are: MonthKRub 12533.036939
MonItemSold 10755.099507
MonAvgTemp 9.570823
dtype: float64

In [47]:
df_MonthSalesTemp.to_csv (r'/dbfs/FileStore/tables/MSC_MonthSalesTemp.csv', index = None, header=True)



In [48]:
#Check if it works
df_MSC_MonthSalesTemp = pd.read_csv("/dbfs/FileStore/tables/MSC_MonthSalesTemp.csv")

print(df_MSC_MonthSalesTemp.shape)
print(df_MSC_MonthSalesTemp.head(10))

(34, 4)
 Month MonthKRub MonItemSold MonAvgTemp
0 2013-01 32739.59204 47495.0 -9.156667
1 2013-02 33098.88780 46744.0 -4.084735
2 2013-03 38682.50245 54830.0 -7.410921
3 2013-04 26048.29295 41348.0 5.498007
4 2013-05 23351.54330 38878.0 16.610582
5 2013-06 25718.10220 42386.0 19.743669
6 2013-07 23483.18459 40787.0 18.802595
7 2013-08 25248.43192 43818.0 18.296476
8 2013-09 42016.35750 49326.0 10.109722
9 2013-10 34760.34540 43529.0 6.241938